In [1]:
import numpy as np
import pandas as pd
import re
import nltk
# nltk.download('punkt')
import warnings
warnings.filterwarnings('ignore')
from gensim.models import Word2Vec

In [2]:
def switch_chude(var):
    switcher = {
                1: "data_final\\bạn bè.txt", 
                2: "data_final\\các câu hỏi phức tạp.txt", 
                3: "data_final\\đất nước.txt",
                4: "data_final\\địa chỉ.txt", 
                5: "data_final\\du lịch.txt", 
                6: "data_final\\gia đình.txt", 
                7: "data_final\\giải trí.txt", 
                8: "data_final\\học tập.txt", 
                9: "data_final\\nghề nghiệp.txt", 
                10: "data_final\\nghi lễ.txt", 
                11: "data_final\\người yêu.txt", 
                12: "data_final\\robot.txt", 
                13: "data_final\\shopping.txt", 
                14: "data_final\\sở thích.txt", 
                15: "data_final\\tán gẫu.txt", 
                16: "data_final\\tdtu.txt", 
                17: "data_final\\thông tin cá nhân.txt", 
                18: "data_final\\trò chuyện về đi ăn.txt"
    }

    return switcher.get(var)

In [3]:
print('1.Bạn bè   2.Các câu hỏi phức tạp   3.Đất nước   4.Địa chỉ   5.Du lịch   6.Gia đình')
print('7.Giải trí   8.Học tập   9.Nghề nghiệp   10.Nghi lễ   11.Người yêu   12.Robot')
print('13.Shopping   14.Sở thích   15.Tán ngẫu   16.TDTU   17.Thông tin cá nhân   18.Trò chuyện về đi ăn\n')

subject = int(input('Chọn chủ đề: '))
while subject<1 or subject>18:
    subject = int(input('Chọn lại chủ đề: '))
    
path = switch_chude(subject)

1.Bạn bè   2.Các câu hỏi phức tạp   3.Đất nước   4.Địa chỉ   5.Du lịch   6.Gia đình
7.Giải trí   8.Học tập   9.Nghề nghiệp   10.Nghi lễ   11.Người yêu   12.Robot
13.Shopping   14.Sở thích   15.Tán ngẫu   16.TDTU   17.Thông tin cá nhân   18.Trò chuyện về đi ăn

Chọn chủ đề: 17


In [4]:
list_sent=[]
with open(path, encoding="utf8") as f:
    train_lines = f.readlines()
    for line in train_lines:
        line = line.split('__eou__')
        line.remove('\n')
        for i in range(len(line)):
            line[i] = line[i].strip()
        list_sent.append(line)   

In [5]:
get_Question = []
for i in list_sent:
    get_Question.append(i[0])

get_Answer = []
for i in list_sent:
    get_Answer.append(i[1])
    
df = pd.DataFrame({"questions": get_Question, "answers": get_Answer})
df

,questions,answers
0,họ và tên của bạn là gì ?,họ và tên của mình là Hà Công Thành
1,bạn bao nhiêu tuổi rồi ?,mình năm nay 22 tuổi
2,bạn đang đi làm hay đang đi học ?,mình đang học tại trường đại học Tôn Đức Thắng
3,bạn cao bao nhiêu ?,mình cao 1 mét 63
4,sở thích của bạn là gì ?,sở thích của mình là chơi game và xem phim
...,...,...
770,Bạn cho mình hỏi bạn tên gì ?,Hiếu
771,Bạn bao nhiêu tuổi ?,22
772,Bạn cho mình hỏi bạn là nam hay nữ ?,nam
773,Bạn cho mình hỏi bạn học ở đâu ?,TDTU


# Word2vec

In [6]:
#list of libraries used by the code
import string
import gensim
from gensim.models import Word2Vec
import math
import random
from math import *

def tokenizeWord(sentences):
  sents=[]
  for sentence in sentences:
    newSent = sentence.lower()
    newSent = re.sub(r"[\.,\?]+$", "", newSent)
    tokens = nltk.word_tokenize(newSent)
    sents.append(tokens)
  return sents

tokenizeQuestion = tokenizeWord(df['questions'].tolist())
tokenizeAnswer = tokenizeWord(df['answers'].tolist())
tokenize = []
tokenize.extend(tokenizeQuestion)
tokenize.extend(tokenizeAnswer)

modelw2v = Word2Vec(sentences=tokenize, vector_size=100, window=5, sg=1, min_count=2)

def cosine_similarity1(v1, v2):
  num = np.dot(v1, v2)
  d1 = np.dot(v1, v1)
  d2 = np.dot(v2, v2)
  if d1 > 0.0 and d2 > 0.0:
    return num / math.sqrt(d1 * d2)
  else:
    return 0.0

def embedding_word(words):
  vectors = [modelw2v.wv[word] for word in words if word in modelw2v.wv]
  if vectors:
    return np.average(vectors, axis=0)
  else:
    return np.zeros(modelw2v.wv.vector_size)

def compare_question(question_input, question_in_df):
  q_input = {word for word in gensim.utils.simple_preprocess(question_input)}
  q_in_df = {word for word in gensim.utils.simple_preprocess(question_in_df) }
  return cosine_similarity1(embedding_word(q_input), embedding_word(q_in_df))

def getAns(question_input):
    get_cosime = []
    for i in range(len(get_Question)):
        get_cosime.append(compare_question(question_input, get_Question[i]))

    result = max(get_cosime)

    ans=[]
    for i in range(len(get_cosime)):
        if get_cosime[i] == result:
            ans.append(get_Answer[i])

    print("Word2Vec: ",random.choice(ans))

# TF-IDF

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(df['questions'].tolist())

from sklearn.metrics.pairwise import linear_kernel
def getCosineSimilarity(q1, q2):
  tfidf_q1 = vectorizer.transform([q1])
  tfidf_q2 = vectorizer.transform([q2])
  return linear_kernel(tfidf_q1, tfidf_q2).flatten()[0]

def getAnswer(q):
  max_score = 0
  answer = ''
  for idx, row in df.iterrows():
    score = getCosineSimilarity(q.lower(), row['questions'])
    if score > max_score:
      max_score = score
      answer = row["answers"]
  print("TF-IDF: ",answer)

# Input and print

In [8]:
while(True):
    stringInput=str(input('Nhập câu: '))
    if(stringInput.lower() == 'thoat' or stringInput.lower() == 'thoát'):
        break
    getAns(stringInput)
    getAnswer(stringInput)
    print("\n")

Nhập câu: thoat


In [13]:
q_input = {word for word in gensim.utils.simple_preprocess("người yêu chưa?")}
print(q_input)

{'yêu', 'người', 'chưa'}


In [14]:
for word in q_input:
    if word in modelw2v.wv:
        print(modelw2v.wv[word])

[-0.03782536  0.0517998   0.06221983 -0.00494829 -0.06299109 -0.28582203
 -0.04181548  0.23627687 -0.0694603  -0.11514656 -0.10541163 -0.19475819
 -0.08800478 -0.05920701  0.10077775 -0.1353102   0.01631377 -0.18018511
 -0.01792094 -0.32390893 -0.00255154  0.11324304  0.18807998 -0.08453412
 -0.0162452   0.03139582 -0.18880665  0.02418373 -0.11717433  0.01045646
  0.27076575  0.0728436   0.1586123  -0.16893718 -0.09163054  0.08328474
  0.02327818 -0.05708412 -0.05641179 -0.15720096 -0.0447504  -0.11325675
 -0.08757935 -0.06104799  0.07068398 -0.1371267  -0.16750893 -0.03028401
  0.14855859  0.0925623   0.05187007  0.00499872  0.06836873  0.06817614
 -0.00830157  0.11802836  0.10547975 -0.02276921 -0.09524318  0.2048295
  0.04856482 -0.03295986 -0.04968133 -0.05766934 -0.08380578  0.11675487
  0.01704728  0.14228927 -0.15982138  0.24900386 -0.08245738  0.09234434
  0.16712289 -0.07558048  0.1861952   0.06477365  0.04248075 -0.0353619
 -0.02813539  0.00424471 -0.10157917  0.02374456 -0.0

In [18]:
tfidf_q1 = vectorizer.transform(["mày ăn cơm chưa?"])
tfidf_q2 = vectorizer.transform(["quê mày ở đâu?"])
print(linear_kernel(tfidf_q1, tfidf_q2).flatten()[0])

[[0.41277914]]


In [ ]:
0.41277914